In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, log_loss

In [25]:
cc_apps=pd.read_csv('../input/credit-card-dataset/credict.csv')
cc_apps.columns=['Gender', 'Age', 'Debt', 'Married', 'BankCustomer', 'Education Level', 'Ethnicity', 'YearsEmployed', 'PriorDefault', 'Employed', 'CreditScore', 'Drivers License', 'Citizen','ZipCode', 'Income','ApprovalStatus']
cc_apps=cc_apps.replace('?','NaN')
cc_apps=cc_apps.replace(['-','+'],[0,1])
cc_apps=cc_apps.astype({'Age':float})
cc_apps

In [26]:
x=cc_apps.drop(['ApprovalStatus','Drivers License','ZipCode'],axis=1)
y=cc_apps.ApprovalStatus

In [27]:
object_cols=x.drop(['Age','Debt','YearsEmployed','CreditScore',"Income"],axis=1)
num_cols=x.select_dtypes(exclude=['object'])

In [28]:
object_cols_train, object_cols_valid, num_cols_train,num_cols_valid=train_test_split(object_cols,num_cols,test_size=0.3,train_size=0.7,random_state=0)

In [29]:
imputer=SimpleImputer()
imp_num_train=pd.DataFrame(imputer.fit_transform(num_cols_train))
imp_num_valid=pd.DataFrame(imputer.fit_transform(num_cols_valid))
imp_num_train.columns=num_cols_train.columns
imp_num_valid.columns=num_cols_valid.columns

encode=OrdinalEncoder()
enc_obj_train=encode.fit_transform(object_cols_train)
enc_obj_valid=encode.transform(object_cols_valid)
enc_obj_train=pd.DataFrame(enc_obj_train, columns=object_cols_train.columns)
enc_obj_valid=pd.DataFrame(enc_obj_valid, columns=object_cols_valid.columns)


In [92]:
y_train,y_valid=train_test_split(y, test_size=0.3, train_size=0.7, random_state=0)
x_train=pd.concat([enc_obj_train,imp_num_train],axis=1)
x_valid=pd.concat([enc_obj_valid, imp_num_valid], axis=1)

scaler=MinMaxScaler(feature_range=(0,2))
scaledx_train=scaler.fit_transform(x_train)
scaledx_valid=scaler.transform(x_valid)

In [93]:
model=LogisticRegression()
model.fit(scaledx_train,y_train)
pred=model.predict(scaledx_valid)
print('Confusin Matrix : \n',confusion_matrix(y_valid,pred))
print('\nClassification Report: \n',classification_report(y_valid, pred))
print('\nAccuracy:\n',accuracy_score(y_valid,pred))
print('\nLog loss:\n',log_loss(y_valid,pred))